### Module library

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from mlxtend.preprocessing import TransactionEncoder

import time
start_time = time.time()

### Baca dataset mentah

In [2]:
df= pd.read_csv('../data/raw_data.csv')
df

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,5937688909,4978118542,False,1618056811,947,5384.0,2.0,7,22,80,5,155,"19,120,107,79,94","31,49,70,110,25"
1,5937680202,4978115238,True,1618056560,1116,3938.0,3.0,7,22,64,8,171,"128,5,8,59,112","85,6,56,42,45"
2,5937677706,4978115412,True,1618056484,1245,4343.0,2.0,7,22,52,5,151,"135,70,104,59,31","58,44,107,46,87"
3,5937677414,4978110742,False,1618056474,1019,3626.0,3.0,7,22,57,7,274,"20,71,82,114,66","26,46,89,129,14"
4,5937677106,4978110923,True,1618056464,1082,3216.0,3.0,7,22,43,7,227,"26,8,104,87,11","73,70,135,42,67"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622165,6086174414,5094955187,False,1626151320,1882,3011.0,2.0,7,22,41,6,153,"30,76,123,47,41","88,48,58,69,84"
622166,6086173100,5094955069,True,1626151212,1977,2173.0,1.0,7,22,34,3,251,"25,42,5,3,98","109,39,129,27,14"
622167,6086172017,5094955078,False,1626151108,2064,3918.0,3.0,7,22,54,4,251,"26,14,48,52,19","107,101,1,28,111"
622168,6086170106,5094954883,False,1626150969,2003,4334.0,5.0,7,22,77,6,241,"46,61,135,79,14","27,86,104,52,44"


In [3]:
# eksplorasi data dengan Profiling
# from pandas_profiling import ProfileReport
# ProfileReport(df)

#### deskripsi dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622170 entries, 0 to 622169
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   match_id       622170 non-null  int64  
 1   match_seq_num  622170 non-null  int64  
 2   radiant_win    622170 non-null  bool   
 3   start_time     622170 non-null  int64  
 4   duration       622170 non-null  int64  
 5   avg_mmr        622170 non-null  float64
 6   num_mmr        622170 non-null  float64
 7   lobby_type     622170 non-null  int64  
 8   game_mode      622170 non-null  int64  
 9   avg_rank_tier  622170 non-null  int64  
 10  num_rank_tier  622170 non-null  int64  
 11  cluster        622170 non-null  int64  
 12  radiant_team   622170 non-null  object 
 13  dire_team      622170 non-null  object 
dtypes: bool(1), float64(2), int64(9), object(2)
memory usage: 62.3+ MB


### Missing data

In [5]:
df.isnull().sum()

match_id         0
match_seq_num    0
radiant_win      0
start_time       0
duration         0
avg_mmr          0
num_mmr          0
lobby_type       0
game_mode        0
avg_rank_tier    0
num_rank_tier    0
cluster          0
radiant_team     0
dire_team        0
dtype: int64

### Menyaring data serta memilih atribut dan label yang digunakan

In [6]:
df = df[(df['avg_mmr'] > 1999) & (df['avg_mmr'] < 7001)]

In [7]:
df= df[['radiant_team','dire_team','radiant_win']]
df

,radiant_team,dire_team,radiant_win
0,"19,120,107,79,94","31,49,70,110,25",False
1,"128,5,8,59,112","85,6,56,42,45",True
2,"135,70,104,59,31","58,44,107,46,87",True
3,"20,71,82,114,66","26,46,89,129,14",False
4,"26,8,104,87,11","73,70,135,42,67",True
...,...,...,...
622165,"30,76,123,47,41","88,48,58,69,84",False
622166,"25,42,5,3,98","109,39,129,27,14",True
622167,"26,14,48,52,19","107,101,1,28,111",False
622168,"46,61,135,79,14","27,86,104,52,44",False


### Shuffle data

In [8]:
from sklearn.utils import shuffle
df = shuffle(df)
df = df.reset_index(drop=True)
df

,radiant_team,dire_team,radiant_win
0,"18,13,23,21,1","42,105,81,94,84",False
1,"13,49,94,110,112","81,91,121,74,72",True
2,"121,104,106,71,48","2,27,93,74,110",True
3,"44,9,54,74,119","72,104,30,7,35",True
4,"74,44,123,39,50","121,19,13,61,8",False
...,...,...,...
621059,"35,84,45,120,8","74,70,50,104,27",False
621060,"6,55,71,10,98","120,107,54,76,86",False
621061,"59,70,135,26,87","28,9,34,93,50",True
621062,"129,83,56,44,19","114,40,37,86,46",True


### Balancing data

In [9]:
# print("Before balancing :")
# print(df['radiant_win'].value_counts())

# row_drop = []
# for i in range(len(df)):
    
#     if df['radiant_win'][i] == True:
#         row_drop.append(i)
    
#     if len(row_drop) >= 95082:
#         break;

# df = df.drop(labels=row_drop, axis=0) 

# print()
# print("After balancing :")
# print(df['radiant_win'].value_counts())

Before balancing :
True     358073
False    262991
Name: radiant_win, dtype: int64

After balancing :
False    262991
True     262991
Name: radiant_win, dtype: int64


### Memecah kolom radiant_team dan dire_team menjadi numpy arry

In [10]:
raw_input = df['radiant_team'] + ',' + df['dire_team']
list_input = list(raw_input.apply(lambda x:x.split(',')))
list_target = list(df['radiant_win'].apply(lambda x: 1 if x == True else 0))

### Membuat dataset baru dengan 1 atribut per hero (121 atribut dan 1 label)

In [11]:
data_count = len(list_input)
data_count

525982

In [12]:
TE = TransactionEncoder()
input_te_data = TE.fit(list_input).transform(list_input)
df_input = pd.DataFrame(input_te_data, columns=TE.columns_)
df_input = df_input.drop([ '0'], axis=1)
df_input

,0,1,10,100,101,102,103,104,105,106,...,90,91,92,93,94,95,96,97,98,99
0,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,True,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525977,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
525978,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
525979,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
525980,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
df_input = df_input.apply(lambda x : x*0 if False else x*1)
df_input

,0,1,10,100,101,102,103,104,105,106,...,90,91,92,93,94,95,96,97,98,99
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525977,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
525978,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
525979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
525980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
columns_name = df_input.columns.to_list()
columns_name[0]

for row in range(df_input.shape[0]):
    for col in columns_name:
        if df_input[col][row] == 1:
            if col in list_input[row][5:]:
                df_input[col][row] = -1

In [15]:
dataset = df_input
dataset['radiant_win'] = list_target

In [16]:
dataset.isnull().sum().sum()

0

In [17]:
dataset = dataset.astype('int')

In [18]:
dataset

,0,1,10,100,101,102,103,104,105,106,...,91,92,93,94,95,96,97,98,99,radiant_win
0,0,1,0,0,0,0,0,0,-1,0,...,0,0,0,-1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,-1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525977,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
525978,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
525979,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,0,0,0,0,0,0,1
525980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Menyimpan datasets bersih

In [19]:
dataset.to_csv('clean_data.csv', sep=',')

In [20]:
ex_time = time.time() - start_time
print("--- %s menit ---" % (ex_time/60))

--- 34.323275566101074 menit ---
